#### Importiamo le librerie necessarie. Useremo ResNet50.

In [ ]:
from tensorflow import keras
import numpy as np
import tarfile
from time import time
from tensorflow.keras.applications import resnet50
import matplotlib.pyplot as plt
import pandas as pd

import pathlib

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf

from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.applications import InceptionV3
from IPython import display

import matplotlib.pyplot as plt
import numpy as np

## Creiamo train e test set

In [ ]:
import os 
import shutil


data_dir = pathlib.Path(r"C:\Users\feder\Desktop\AwA2-data\Animals_with_Attributes2\JPEGImages")
species = ["horse", "sheep"]
n_species= len(species)

filenames = tf.io.gfile.glob(str(data_dir) + '/*/*')
new_filenames=[]
for filename in filenames:
    if any(substring in filename for substring in species) == True:
        new_filenames.append(filename)
    else:
        continue

num_samples = len(new_filenames)

filesToFind = tf.random.shuffle(new_filenames)

# Had an issue here but needed to define and then reference the filename variable itself for filename in os.listdir(folderPath):
train_files = filesToFind[:int(num_samples*0.8)]
val_files = filesToFind[int(num_samples*0.8): int(num_samples*0.9)]
test_files = filesToFind[-int(num_samples*0.1):]



In [ ]:
dest_train = pathlib.Path(r"C:\Users\feder\Desktop\AwA2-data\Animals_with_Attributes2\train\\")
dest_val = pathlib.Path(r"C:\Users\feder\Desktop\AwA2-data\Animals_with_Attributes2\val\\")
dest_test = pathlib.Path(r"C:\Users\feder\Desktop\AwA2-data\Animals_with_Attributes2\test\\")

if os.path.isdir(dest_train):
    print("Exists")
else:
    print("Doesn't exists")
    os.mkdir(dest_train)
if os.path.isdir(dest_val):
    print("Exists")
else:
    print("Doesn't exists")
    os.mkdir(dest_val)
if os.path.isdir(dest_test):
    print("Exists")
else:
    print("Doesn't exists")
    os.mkdir(dest_test)

for filename in train_files: 
    mystr = filename.numpy().decode("UTF-8")
    dest = str(dest_train)+"\\"+mystr[mystr.rfind("\\")+1 : mystr.rfind("_")]
    print(dest)
    if os.path.isdir(dest):
        print("Exists")
    else:
        print("Doesn't exists")
        os.mkdir(dest)
    shutil.copy(mystr, dest)
    
for filename in val_files: 
    mystr = filename.numpy().decode("UTF-8")
    dest = str(dest_val)+"\\"+mystr[mystr.rfind("\\")+1 : mystr.rfind("_")]
    if os.path.isdir(dest):
        print("Exists")
    else:
        print("Doesn't exists")
        os.mkdir(dest)
    shutil.copy(mystr, dest)
    
for filename in test_files: 
    mystr = filename.numpy().decode("UTF-8")
    dest = str(dest_test)+"\\"+mystr[mystr.rfind("\\")+1 : mystr.rfind("_")]
    if os.path.isdir(dest):
        print("Exists")
    else:
        print("Doesn't exists")
        os.mkdir(dest)
    shutil.copy(mystr, dest)


## Prima configurazione

In [ ]:
train_processing = keras.preprocessing.image.ImageDataGenerator(preprocessing_function= resnet50.preprocess_input,
                                                                shear_range=0.4,
                                                                zoom_range=0.4,
                                                                horizontal_flip=True,
                                                                rotation_range=30,
                                                                width_shift_range=0.3,
                                                                height_shift_range=0.3,
                                                                brightness_range=[0.5, 1.5])


train_generator = train_processing.flow_from_directory(directory = r"C:\Users\feder\Desktop\AwA2-data\Animals_with_Attributes2\train",
                                    target_size = (224, 224),
                                    color_mode = "rgb",
                                    batch_size = 32,
                                    class_mode = "categorical",
                                    shuffle = True,
                                    seed = 1234)

#### Ora andiamo a definire il test generator, in cui non applicheremo alcun tipo di data agumentation ovviamente, in quanto i dati di test devono essere utilizzati come sono.

In [ ]:
val_processing = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=resnet50.preprocess_input)

val_generator = test_processing.flow_from_directory(
        directory=r"C:\Users\feder\Desktop\AwA2-data\Animals_with_Attributes2\val",
        target_size=(224, 224),
        color_mode="rgb",
        batch_size=32,
        class_mode="categorical",
        shuffle=True
)

In [ ]:
test_processing = keras.preprocessing.image.ImageDataGenerator(preprocessing_function=resnet50.preprocess_input)

test_generator = test_processing.flow_from_directory(
        directory=r"C:\Users\feder\Desktop\AwA2-data\Animals_with_Attributes2\test",
        target_size=(224, 224),
        color_mode="rgb",
        batch_size=32,
        class_mode="categorical",
        shuffle=True
)

#### Importiamo il modello con i pesi adatti (imagenet), e settiamo i layer iniziali come non allenabili

In [ ]:
base_net = resnet50.ResNet50(weights="imagenet", include_top=False,
	input_shape=(224, 224, 3), pooling="avg")

In [ ]:
for layer in base_net.layers:
    layer.trainable = False

#### Ora aggiungiamo due fully connected layers e poi creiamo un nuovo modello "net", dato dalla combinazione dei due 

In [ ]:
# Output of the base_net model
x = base_net.output
# intermediate fully-connected layer + ReLU
x = keras.layers.Dense(1024, activation='relu')(x)
# final fully-connected layer + SoftMax 
pred = keras.layers.Dense(n_species, activation='softmax')(x)

In [ ]:
net1 = keras.Model(inputs=base_net.input, outputs=pred)

#### Compiliamo il modello usando come loss la categorical crossentropy, come optimizer RMSprop e come metrica l'accuracy

In [ ]:
net1.compile(loss=keras.losses.categorical_crossentropy,
            optimizer=keras.optimizers.RMSprop(),
            metrics=['accuracy'])

#### Ora fittiamo il modello con 10 epoche e visualizziamo i risultati in termini di accuracy e loss su train e test set

In [ ]:
history1 = net1.fit(train_generator,
          epochs=10,
          validation_data=val_generator)

In [ ]:
def accuracy_loss(history):
    plt.figure(figsize=(20,10))
    #
    plt.subplot(1,2,1)
    plt.plot(history.history['acc'], label = "train_accuracy", color = "red")
    plt.plot(history.history['val_acc'], label="train_accuracy", color ="blue")
    plt.legend(loc='best', fontsize = 12)
    plt.xticks(fontsize = 12)
    plt.yticks(fontsize = 12)
    plt.xlabel('Epochs', size = 15)
    plt.ylabel('Accuracy', size = 15)
    #
    plt.subplot(1,2,2)
    plt.plot(history.history['loss'], label = "Train loss", color = "red")
    plt.plot(history.history['val_loss'], label="Test loss", color = "blue")
    plt.legend(loc='best', fontsize = 12)
    plt.xticks(fontsize = 12)
    plt.yticks(fontsize = 12)
    plt.xlabel('Epochs', size = 15)
    plt.ylabel('Loss', size = 15)

In [ ]:
performance1 = net1.evaluate(test_generator)

In [ ]:
accuracy_loss(history1)